In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from tkinter.filedialog import askopenfiles

In [2]:
files = askopenfiles(title="Select files", filetypes=[("Image files", "*.png")], multiple=True)

In [3]:
def getFeatures(image, binary_mask):
    features = {}

    hist_b = cv2.calcHist([image], [0], binary_mask, [256], [0, 256])
    hist_g = cv2.calcHist([image], [1], binary_mask, [256], [0, 256])
    hist_r = cv2.calcHist([image], [2], binary_mask, [256], [0, 256])

    hist_b = hist_b.flatten()
    hist_g = hist_g.flatten()
    hist_r = hist_r.flatten()
    hists = [hist_b, hist_g, hist_r]

    col_b = hist_b.argmax() / hist_b.shape[0]
    col_g = hist_g.argmax() / hist_g.shape[0]
    col_r = hist_r.argmax() / hist_r.shape[0]

    features["blue"] = col_b
    features["green"] = col_g
    features["red"] = col_r

        
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        real_area = cv2.contourArea(largest_contour)
        
        _, radius = cv2.minEnclosingCircle(largest_contour)
        radius = int(radius)
        circle_area = np.pi * radius ** 2

        rect = cv2.minAreaRect(largest_contour)
        box = cv2.boxPoints(rect)
        box = np.int64(box)
        rect_area = cv2.contourArea(box)
        
        
        circle_area_ratio = real_area / circle_area
        rect_area_ratio = real_area / rect_area
        
        total_area_ratio = real_area / (image.shape[0] * image.shape[1])
        
        features["circle_area_ratio"] = circle_area_ratio
        features["rect_area_ratio"] = rect_area_ratio
        features["total_area_ratio"] = total_area_ratio
        
        
    else:
        raise ValueError("No contours found in binary mask")
    
    return features


In [8]:
all_features = []
for file in files:
    if file.name.endswith(".mask.png"):
        continue
    image = cv2.imread(file.name)
    keyimage = cv2.imread(file.name.replace(".png", ".mask.png"), cv2.IMREAD_GRAYSCALE)
    binary_mask = (keyimage // 255).astype(np.uint8)

    features = getFeatures(image, binary_mask)
    all_features.append(features)
all_features = np.array(all_features)

In [9]:
import csv

with open("gruenglas.csv", mode='w', newline='') as file:
    # Create a writer object with fieldnames based on the keys of the first dictionary
    fieldnames = ['index'] + list(all_features[0].keys())
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    
    # Write the header (field names)
    writer.writeheader()
    
    # Write the data rows
    writer.writerows(all_features)